In [1]:
from bokeh.io import output_notebook, show, push_notebook
from bokeh.plotting import figure
from bokeh.layouts import gridplot
import numpy as np
import random
import time

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
min_screen_x, min_screen_y, max_screen_x, max_screen_y, me_x, me_y, target_x, target_y = \
            0, 0, 200, 200, 20, 20, 50, 60
plt_loc = figure(
    plot_width=600,
    plot_height=600,
    toolbar_location=None,
    x_range=(min_screen_x-30, max_screen_x+30),
    y_range=(max_screen_y+30, min_screen_y-30),
    x_axis_location="above",
    title="敌我距离")  # use up-left corner as origin
plt_loc.title.align = "center"
plt_loc.title.text_color = "orange"
plt_loc.title.text_font_size = "25px"
plt_loc.title.background_fill_color = "blue"

rd_loc = plt_loc.circle(
            [me_x, target_x], [me_y, target_y],
            size=[50, 20],
            line_color="gold",
            line_width=[10, 1],
            fill_color=["green", "yellow"],
            fill_alpha=0.6)

In [4]:
location = {
    'me_x': 1,
    'me_y': 2,
    'target_x': 2,
    'target_y':3
}
me_speed = 5
target_speed=15

def calc_eucl_dist(pos1, pos2):
    # calc Euclidean Distance
    _coords1 = np.array(pos1)  # location of me
    _coords2 = np.array(pos2)
    # alternative way: np.linalg.norm(_coords1 - _coords2)
    eucl_dist = np.sqrt(np.sum((_coords1 - _coords2)**2))
    return eucl_dist

def refresh_render(location):
    eucl_dist = calc_eucl_dist((location['target_x'], location['target_y']), (location['me_x'], location['me_y']) )
    plt_loc.title.text = "敌我距离: {:12.2f}".format(eucl_dist)
    # plt_loc.x_range.start = location["min_screen_x"]
    # plt_loc.x_range.end = location["max_screen_x"]
    # plt_loc.y_range.start = location["max_screen_y"]
    # plt_loc.y_range.end = location["min_screen_y"]
    rd_loc.data_source.data['x'] = [
        location['me_x'], location['target_x']
    ]
    rd_loc.data_source.data['y'] = [
        location['me_y'], location['target_y']
    ]
    
    if eucl_dist < 100:
        rd_loc.data_source.data['fill_color']  = ["green", "red"]
    else:
        rd_loc.data_source.data['fill_color']  = ["green", "yellow"]

    push_notebook()  # self.nb_handle

def step(location):
    final_decision = location.copy()
    
    # the target will run out of me as far as possible
    # 要么走x， 要么走y，先看边界，再选距离， 上下左右
    npc = (location['target_x'], location['target_y'])
    player = (location['me_x'], location['me_y'])
    npc_new_loc = action_by_one_side(npc, player, target_speed, mode="run")
    
    final_decision['target_x'], final_decision['target_y'] = npc_new_loc
    
    # samely, for me, run to get more close to target
    player_new_loc = action_by_one_side(player, npc_new_loc, me_speed, mode="catch")
    final_decision['me_x'], final_decision['me_y'] = player_new_loc
    
    
    return final_decision

def action_by_one_side(my_pos, adversary_pos, my_speed, mode="run"):  
    """mode can be run/catch"""
    x, y = my_pos
    available_direction = [
        y - my_speed > min_screen_y,
        y + my_speed < max_screen_y,
        x - my_speed > min_screen_x,
        x + my_speed < max_screen_x,
    ]
    new_location = [
        (x, y - my_speed),
        (x, y + my_speed),
        (x - my_speed, y),
        (x + my_speed, y),
    ]
    
    available_loc = [_l for i, _l in enumerate(new_location) if available_direction[i]]
    
    new_dist = [calc_eucl_dist(my_new_pos, adversary_pos) for my_new_pos in available_loc]
    
    func = max if mode =="run" else min
    best_choice = func(enumerate(new_dist), key=lambda x: x[1])[0]
    my_final_pos = available_loc[best_choice]
    return my_final_pos

In [5]:
nb_handle = show(plt_loc, notebook_handle=True)

In [ ]:
# 把代码放到图下面运行， 看起来方便一点
new_loc = location
for i in range(500):
    new_loc = step(new_loc)
    print(new_loc)

    refresh_render(new_loc)
    time.sleep(0.1)

{'target_y': 18, 'me_x': 1, 'me_y': 7, 'target_x': 2}
{'target_y': 33, 'me_x': 1, 'me_y': 12, 'target_x': 2}
{'target_y': 48, 'me_x': 1, 'me_y': 17, 'target_x': 2}
{'target_y': 63, 'me_x': 1, 'me_y': 22, 'target_x': 2}
{'target_y': 78, 'me_x': 1, 'me_y': 27, 'target_x': 2}
{'target_y': 93, 'me_x': 1, 'me_y': 32, 'target_x': 2}
{'target_y': 108, 'me_x': 1, 'me_y': 37, 'target_x': 2}
{'target_y': 123, 'me_x': 1, 'me_y': 42, 'target_x': 2}
{'target_y': 138, 'me_x': 1, 'me_y': 47, 'target_x': 2}
{'target_y': 153, 'me_x': 1, 'me_y': 52, 'target_x': 2}
{'target_y': 168, 'me_x': 1, 'me_y': 57, 'target_x': 2}
{'target_y': 183, 'me_x': 1, 'me_y': 62, 'target_x': 2}
{'target_y': 198, 'me_x': 1, 'me_y': 67, 'target_x': 2}
{'target_y': 198, 'me_x': 1, 'me_y': 72, 'target_x': 17}
{'target_y': 198, 'me_x': 1, 'me_y': 77, 'target_x': 32}
{'target_y': 198, 'me_x': 1, 'me_y': 82, 'target_x': 47}
{'target_y': 198, 'me_x': 1, 'me_y': 87, 'target_x': 62}
{'target_y': 198, 'me_x': 1, 'me_y': 92, 'target_x'

{'target_y': 18, 'me_x': 21, 'me_y': 27, 'target_x': 2}
{'target_y': 3, 'me_x': 21, 'me_y': 22, 'target_x': 2}
{'target_y': 18, 'me_x': 16, 'me_y': 22, 'target_x': 2}
{'target_y': 3, 'me_x': 16, 'me_y': 17, 'target_x': 2}
{'target_y': 18, 'me_x': 11, 'me_y': 17, 'target_x': 2}
{'target_y': 33, 'me_x': 11, 'me_y': 22, 'target_x': 2}
{'target_y': 48, 'me_x': 11, 'me_y': 27, 'target_x': 2}
{'target_y': 63, 'me_x': 11, 'me_y': 32, 'target_x': 2}
{'target_y': 78, 'me_x': 11, 'me_y': 37, 'target_x': 2}
{'target_y': 93, 'me_x': 11, 'me_y': 42, 'target_x': 2}
{'target_y': 108, 'me_x': 11, 'me_y': 47, 'target_x': 2}
{'target_y': 123, 'me_x': 11, 'me_y': 52, 'target_x': 2}
{'target_y': 138, 'me_x': 11, 'me_y': 57, 'target_x': 2}
{'target_y': 153, 'me_x': 11, 'me_y': 62, 'target_x': 2}
{'target_y': 168, 'me_x': 11, 'me_y': 67, 'target_x': 2}
{'target_y': 183, 'me_x': 11, 'me_y': 72, 'target_x': 2}
{'target_y': 198, 'me_x': 11, 'me_y': 77, 'target_x': 2}
{'target_y': 198, 'me_x': 11, 'me_y': 82, '

{'target_y': 3, 'me_x': 21, 'me_y': 22, 'target_x': 2}
{'target_y': 18, 'me_x': 16, 'me_y': 22, 'target_x': 2}
{'target_y': 3, 'me_x': 16, 'me_y': 17, 'target_x': 2}
{'target_y': 18, 'me_x': 11, 'me_y': 17, 'target_x': 2}
{'target_y': 33, 'me_x': 11, 'me_y': 22, 'target_x': 2}
{'target_y': 48, 'me_x': 11, 'me_y': 27, 'target_x': 2}
{'target_y': 63, 'me_x': 11, 'me_y': 32, 'target_x': 2}
{'target_y': 78, 'me_x': 11, 'me_y': 37, 'target_x': 2}
{'target_y': 93, 'me_x': 11, 'me_y': 42, 'target_x': 2}
{'target_y': 108, 'me_x': 11, 'me_y': 47, 'target_x': 2}
{'target_y': 123, 'me_x': 11, 'me_y': 52, 'target_x': 2}
{'target_y': 138, 'me_x': 11, 'me_y': 57, 'target_x': 2}
{'target_y': 153, 'me_x': 11, 'me_y': 62, 'target_x': 2}
{'target_y': 168, 'me_x': 11, 'me_y': 67, 'target_x': 2}
{'target_y': 183, 'me_x': 11, 'me_y': 72, 'target_x': 2}
{'target_y': 198, 'me_x': 11, 'me_y': 77, 'target_x': 2}
{'target_y': 198, 'me_x': 11, 'me_y': 82, 'target_x': 17}
{'target_y': 198, 'me_x': 11, 'me_y': 87,